# Assistant APIs with Code Interpreter for Audit

In [95]:
import os
import json
import requests
import time
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv() 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key= os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-05-01-preview"
)




In [9]:
file = client.files.create(file=open("data.csv", "rb"), purpose="assistants")
file

FileObject(id='assistant-XMHdmyW3OM60Mi43qhKZUX58', bytes=1783, created_at=1730451786, filename='data.csv', object='file', purpose='assistants', status='processed', status_details=None)

In [22]:
file.id

'assistant-XMHdmyW3OM60Mi43qhKZUX58'

In [20]:
assistant = client.beta.assistants.retrieve(test.id)

In [23]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "are there any outliers?",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [26]:
# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
  print(run.status)


run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

If you need any further analysis or assistance, please let me know!


In [38]:
metaprompt = """"
You are a Risk Detector assistant. You are specialized in running code against provided files to identify anomalies and outliers.

"""

In [39]:
assistant = client.beta.assistants.create(
  model="gpt-4o", # replace with model deployment name.
  instructions= metaprompt,
  tools=[{"type":"code_interpreter"}],
  tool_resources={"code_interpreter":{"file_ids":[file.id]}},
  temperature=1,
  top_p=1
)

In [40]:
# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="are there any outliers?" # Replace this with your prompt
)


# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
  print(run.status)


The following outliers were identified in the dataset:

**Revenue (USD):**
```
    ID      Company  Revenue (USD)  Expenses (USD)  Profit (USD)
 5    6    Zeta Corp      1500000.0       1200000.0      300000.0
 17  18   Sigma Corp      1800000.0       1500000.0      300000.0
 24  25  Alpha2 Corp      2500000.0       2000000.0      500000.0
 29  30   Zeta2 Corp      1200000.0       1000000.0      200000.0
 33  34  Kappa2 Corp      2200000.0       1700000.0      500000.0
 39  40     Pi2 Ltd.      3000000.0       2500000.0      500000.0
```

**Expenses (USD):**
```
    ID      Company  Revenue (USD)  Expenses (USD)  Profit (USD)
 5    6    Zeta Corp      1500000.0       1200000.0      300000.0
 17  18   Sigma Corp      1800000.0       1500000.0      300000.0
 24  25  Alpha2 Corp      2500000.0       2000000.0      500000.0
 29  30   Zeta2 Corp      1200000.0       1000000.0      200000.0
 33  34  Kappa2 Corp      2200000.0       1700000.0      500000.0
 39  40     Pi2 Ltd.      3000000.0 

In [37]:
messages.data

[Message(id='msg_g1NQSt3XqeeryqsqZAeYs1rd', assistant_id='asst_1HPxDrzB7v30dFSJaDmtUfEb', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='If you need any further analysis or assistance, please let me know!'), type='text')], created_at=1730452584, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_fOiFcSpaTOJG4cQjXdaEa7VV', status=None, thread_id='thread_N7BqNs8oZDlCfxUOIxUsipDX'),
 Message(id='msg_ZXSFV2Wl8ZNqXPwBb2ZU6mVr', assistant_id='asst_1HPxDrzB7v30dFSJaDmtUfEb', attachments=[Attachment(file_id='assistant-CbpQ0Ff0zDuHYwhYrmzouMyA', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=137, file_path=FilePath(file_id='assistant-CbpQ0Ff0zDuHYwhYrmzouMyA'), start_index=107, text='sandbox:/mnt/data/outliers.csv', type='file_path')], value='The table with outliers has been saved. Y

In [36]:
# Assuming 'messages_page' is your SyncCursorPage[Message] object
file_ids = []

for message in messages.data:
    # Check if the message has attachments
    if hasattr(message, 'attachments') and message.attachments:
        for attachment in message.attachments:
            # Check if the attachment has a 'file_id' attribute
            if hasattr(attachment, 'file_id'):
                file_ids.append(attachment.file_id)

# Now 'file_ids' contains all the extracted file IDs
print("Extracted file IDs:", file_ids)


Extracted file IDs: ['assistant-CbpQ0Ff0zDuHYwhYrmzouMyA', 'assistant-XMHdmyW3OM60Mi43qhKZUX58']


In [27]:
def get_file_ids_from_thread(client, thread_id):
    """
    Extracts file_ids from the attachments of messages in a thread, if attachments are not empty.

    :param client: The client object used to interact with the API.
    :param thread_id: The ID of the thread to extract messages from.
    :return: A list of file_ids extracted from the attachments of the messages.
    """
    # Get the list of messages from the thread
    messages = [i for i in client.beta.threads.messages.list(thread_id=thread_id)]

    # Extract file_ids from each message's attachments if attachments are not empty
    file_ids = [
        attachment.file_id
        for message in messages
        if message.attachments
        for attachment in message.attachments
    ]
    
    return file_ids

# Example usage
file_ids = get_file_ids_from_thread(client, thread_id='thread_12yh3C890jvQYzTsutTUc4rh')
print(file_ids)  # Output will be a list of file_ids from all messages with attachments


['assistant-o6QzcwtUgZJm7aj8NWXsdwVD']


In [99]:
content = client.files.content(file_ids[0]).read()
content

b'ID,Company,Revenue (USD),Expenses (USD),Profit (USD)\n6,Zeta Corp,1500000.0,1200000.0,300000.0\n18,Sigma Corp,1800000.0,1500000.0,300000.0\n25,Alpha2 Corp,2500000.0,2000000.0,500000.0\n30,Zeta2 Corp,1200000.0,1000000.0,200000.0\n34,Kappa2 Corp,2200000.0,1700000.0,500000.0\n40,Pi2 Ltd.,3000000.0,2500000.0,500000.0\n'

In [7]:
def write_file_to_temp_dir(file_id, output_path):
    file_data = client.files.content(file_id)
    file_data_bytes = file_data.read()
    with open(output_path, "wb") as file:
        file.write(file_data_bytes)


some_file_id = file_ids[0]
write_file_to_temp_dir(some_file_id, 'outliers.csv')

In [49]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [66]:
run_steps.data

[RunStep(id='step_tKIWsukUzBEa2oWwwQLxxZ8s', assistant_id='asst_0dhgPrqdSNNMCsFvo7Lcsgoz', cancelled_at=None, completed_at=1730474285, created_at=1730474279, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_krLX6OpCIwXPjaYDxGS3Y9UT', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_YdICRQ1LOgbUY5noT0oNpnA7'), type='message_creation'), thread_id='thread_rIRMFkyfW45t0fensuFDSgaB', type='message_creation', usage=Usage(completion_tokens=648, prompt_tokens=1867, total_tokens=2515), expires_at=None),
 RunStep(id='step_InhMJNvdmbaEZx4o0Y23ncdI', assistant_id='asst_0dhgPrqdSNNMCsFvo7Lcsgoz', cancelled_at=None, completed_at=1730474279, created_at=1730474276, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_krLX6OpCIwXPjaYDxGS3Y9UT', status='completed', step_details=ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call

In [84]:
for i in run_steps.data[3].step_details.tool_calls:
    print(i.code_interpreter.input)

# Remove commas and convert columns to numeric types
data['Revenue (USD)'] = data['Revenue (USD)'].str.replace(',', '').astype(float)
data['Expenses (USD)'] = data['Expenses (USD)'].str.replace(',', '').astype(float)
data['Profit (USD)'] = data['Profit (USD)'].str.replace(',', '').astype(float)

# Display the first few rows to confirm the conversion
data.head()


In [71]:
for idx, step in enumerate(reversed(run_steps.data)):
    print(step.step_details.type)

message_creation
tool_calls
message_creation
tool_calls
message_creation
tool_calls
message_creation


In [90]:
for idx, step in enumerate(reversed(run_steps.data)):
    print(f"**Step {idx+1}:**")
    step_type = step.step_details.type

    # Handle message creation steps
    if step_type == 'message_creation':
        message_id = step.step_details.message_creation.message_id
        # Retrieve the message content
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=message_id
        )
        if message.content:
            for content_piece in message.content:
                if hasattr(content_piece, 'text'):
                    print(content_piece.text.value)
    elif step_type == 'tool_calls':
        tool_calls = step.step_details.tool_calls
        for tool_call in tool_calls:
            code = tool_call.code_interpreter.input
            print(code)
            # Display any error messages
            if hasattr(tool_call.code_interpreter, 'error'):
                error_message = tool_call.code_interpreter.error.message
                print(f"Code Interpreter Error: {error_message}")
    else:
        print(f"Unknown step type: {step_type}")

**Step 1:**
To identify any outliers in the provided file, I will first need to load the data and inspect its contents. Let's read the file and take a look.
**Step 2:**
import pandas as pd

# Load the data from the uploaded file
file_path = '/mnt/data/assistant-XMHdmyW3OM60Mi43qhKZUX58'
data = pd.read_csv(file_path)

# Display the first few rows and basic information about the data
data_info = data.info()
data_head = data.head()

data_info, data_head
**Step 3:**
The dataset contains 40 entries with the following columns: 'ID', 'Company', 'Revenue (USD)', 'Expenses (USD)', and 'Profit (USD)'. It appears that the columns 'Revenue (USD)', 'Expenses (USD)', and 'Profit (USD)' are stored as object (string) types because they contain commas as thousand separators.

To identify outliers, I will first need to convert these columns to numeric types. I'll remove the commas and convert the columns to numeric data types, then I will proceed with detecting any outliers.

Let's perform these steps n

In [65]:
print(run_steps.data[1].step_details.tool_calls[0].code_interpreter.input)

def identify_outliers(data, column):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Calculate the lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

# Identify outliers in 'Revenue (USD)', 'Expenses (USD)', and 'Profit (USD)'
outliers_revenue = identify_outliers(data, 'Revenue (USD)')
outliers_expenses = identify_outliers(data, 'Expenses (USD)')
outliers_profit = identify_outliers(data, 'Profit (USD)')

outliers_revenue, outliers_expenses, outliers_profit


In [91]:
! pip install azure-storage-blob

In [96]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient


connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [100]:
blob_client = blob_service_client.get_blob_client(container='anomalies', blob='outliers.csv')
blob_client.upload_blob(content, overwrite=True)

{'etag': '"0x8DCFB40B78F6D2E"',
 'last_modified': datetime.datetime(2024, 11, 2, 13, 17, 35, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\xb3\x18^\xdc\x93Q\x9e\xc6`\x1a\x89\xebF\xd1\xa6\xa3'),
 'client_request_id': 'd2dae111-991c-11ef-9181-010101010000',
 'request_id': '0fa4d28d-e01e-0059-5329-2d2bbe000000',
 'version': '2024-11-04',
 'version_id': None,
 'date': datetime.datetime(2024, 11, 2, 13, 17, 34, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

In [97]:
blob_client = blob_service_client.get_blob_client(container='anomalies', blob='outliers.csv')
with open(file='outliers.csv', mode="rb") as data:
    blob_client.upload_blob(data, overwrite=True)


# Testing bulk queries

In [9]:
import time

def run_queries(queries, client, assistant_id):
    results = []
    
    for query in queries:
        # Create a thread for each query
        thread = client.beta.threads.create()

        # Add the user query to the thread
        message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=query  # Use the current query from the list
        )

        # Run the assistant with the query
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant_id
        )

        # Looping until the run completes or fails
        while run.status in ['queued', 'in_progress', 'cancelling']:
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )

        # Collect the result once the run is completed
        if run.status == 'completed':
            messages = client.beta.threads.messages.list(
                thread_id=thread.id
            )
            results.append(messages.data[0].content[0].text.value)
        elif run.status == 'requires_action':
            # Handle any required actions if necessary
            results.append("Assistant requires action for query: " + query)
        else:
            # Store the status for failed queries
            results.append(f"Query failed: {run.status} for query: {query}")

    return results


# List of queries to run
queries = [
    "Are there any outliers?",
    "Can you summarize the data trends?",
    "What anomalies do you detect?"
]

# Call the function with the list of queries
results = run_queries(queries, client, assistant.id)

# Output the results for each query
for i, result in enumerate(results):
    print(f"Result for query {i+1}: {result}")


Result for query 1: I have identified the outliers in the dataset. Here are the rows that contain outliers:

| ID | Company     | Revenue (USD) | Expenses (USD) | Profit (USD) |
|----|-------------|---------------|----------------|--------------|
| 6  | Zeta Corp   | 1,500,000     | 1,200,000      | 300,000      |
| 18 | Sigma Corp  | 1,800,000     | 1,500,000      | 300,000      |
| 25 | Alpha2 Corp | 2,500,000     | 2,000,000      | 500,000      |
| 30 | Zeta2 Corp  | 1,200,000     | 1,000,000      | 200,000      |
| 34 | Kappa2 Corp | 2,200,000     | 1,700,000      | 500,000      |
| 40 | Pi2 Ltd.    | 3,000,000     | 2,500,000      | 500,000      |

The file containing these outliers has been saved at: [outliers.csv](sandbox:/mnt/data/outliers.csv)
Result for query 2: Here is a summary of the data trends based on the calculated statistics:

1. **Revenue (USD)**:
   - Mean (average) revenue: $476,625
   - Standard deviation (spread): $726,632.9
   - Minimum revenue: $45,000
   - Max

# Display images

In [ ]:
from PIL import Image
import io

# Assuming response_content is the binary content you're getting from the OpenAI API
response_content = client.files.content(messages.data[0].content[0].image_file.file_id)

# Convert the binary response content to an image
image_bytes = response_content.content
image = Image.open(io.BytesIO(image_bytes))

# Display the image
image.show()




In [57]:
messages

SyncCursorPage[Message](data=[Message(id='msg_WPUgNixVlcYTmKw4W3pKZ3Ce', assistant_id='asst_4aatBVInAyCOhUfzWA5M8yZD', attachments=[Attachment(file_id='assistant-o6QzcwtUgZJm7aj8NWXsdwVD', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=147, file_path=FilePath(file_id='assistant-o6QzcwtUgZJm7aj8NWXsdwVD'), start_index=117, text='sandbox:/mnt/data/outliers.csv', type='file_path')], value='The outliers have been saved to a CSV file. You can download the file using the link below:\n\n[Download outliers.csv](sandbox:/mnt/data/outliers.csv)'), type='text')], created_at=1729577434, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_7NXbrQMJPp73SXurSWDzbfSo', status=None, thread_id='thread_12yh3C890jvQYzTsutTUc4rh'), Message(id='msg_jUNoPpyPnv1iNy5ksR6Faayr', assistant_id='asst_4aatBVInAyCOhUfzWA5M8yZD', attachments=[], complete

In [61]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)



SyncCursorPage[RunStep](data=[RunStep(id='step_AbDJI5I1aABzai99pxuAH9Cc', assistant_id='asst_4aatBVInAyCOhUfzWA5M8yZD', cancelled_at=None, completed_at=1729577435, created_at=1729577434, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_7NXbrQMJPp73SXurSWDzbfSo', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_WPUgNixVlcYTmKw4W3pKZ3Ce'), type='message_creation'), thread_id='thread_12yh3C890jvQYzTsutTUc4rh', type='message_creation', usage=Usage(completion_tokens=37, prompt_tokens=1645, total_tokens=1682), expires_at=None), RunStep(id='step_AYnkm56sje4mZkMtUofdQiTD', assistant_id='asst_4aatBVInAyCOhUfzWA5M8yZD', cancelled_at=None, completed_at=1729577434, created_at=1729577432, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_7NXbrQMJPp73SXurSWDzbfSo', status='completed', step_details=ToolCallsStepDetails(tool_calls=[CodeI

In [81]:
print(run_steps.data[3].step_details.tool_calls[0].code_interpreter.input)

# Convert columns to numeric
data['Revenue (USD)'] = data['Revenue (USD)'].str.replace(',', '').astype(float)
data['Expenses (USD)'] = data['Expenses (USD)'].str.replace(',', '').astype(float)
data['Profit (USD)'] = data['Profit (USD)'].str.replace(',', '').astype(float)

def detect_outliers_iqr(df):
    outliers = pd.DataFrame(columns=df.columns)
    for column in df.select_dtypes(include=[float, int]).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        column_outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        outliers = pd.concat([outliers, column_outliers])

    return outliers

outliers = detect_outliers_iqr(data)

outliers
